In [23]:
# Prep ... import libraries
import pandas as pd
import requests
from pathlib import Path

# Import API key & set base URL
from keys import api_key
url="https://api.walkscore.com/score?format=json&address=skip"

# Define variables
FindState = "HI"

In [29]:
# Import CSV
data_df = pd.read_csv("resources/obesity_data_ALL_states.csv")
data_df = pd.DataFrame(data_df)

# Insert new column at end to import Walk Score
data_df["Walk Score"]=''

# Clean up dataframe by removing any rows with no data
data_df_cleaned=data_df.dropna()

# data_df_cleaned["StateAbbr"].value_counts()

CA    5239
TX    2961
NY    2462
FL    1295
IL    1285
AZ    1000
OH     821
NC     764
MI     724
CO     684
PA     631
IN     514
TN     511
WA     503
VA     499
MA     484
OK     461
GA     444
MO     427
WI     400
LA     369
NV     359
AL     345
OR     336
MN     310
NJ     304
KS     298
KY     262
HI     234
UT     233
CT     220
NE     214
NM     206
MD     199
IA     190
DC     178
SC     167
AR     118
RI      98
MS      75
ID      67
SD      56
AK      55
NH      47
MT      43
ND      25
DE      24
WV      22
ME      20
WY      16
VT      11
Name: StateAbbr, dtype: int64

In [25]:
# Let's do API calls for just ONE state
# We define FindState in first block...
one_state = data_df_cleaned.loc[data_df_cleaned["StateAbbr"] == FindState, :]

# reset index because of dropped rows
one_state.reset_index(drop=True, inplace=True)

one_state.head(10)

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,HI,Honolulu,15003006202,15003,15003006202,44.7,1701,21.338852,-157.875465,
1,HI,Honolulu,15003001700,15003,15003001700,22.9,2437,21.265515,-157.817512,
2,HI,Honolulu,15003005400,15003,15003005400,36.9,1637,21.319959,-157.864110,
3,HI,Honolulu,15003008921,15003,15003008921,19.5,2668,21.404035,-157.991591,
4,HI,Honolulu,15003003300,15003,15003003300,19.8,1132,21.312041,-157.834916,
5,HI,Honolulu,15003000700,15003,15003000700,16.7,2966,21.271340,-157.792699,
6,HI,Honolulu,15003940002,15003,15003940002,34.9,7400,21.394002,-158.124509,
7,HI,Honolulu,15003008613,15003,15003008613,25.8,904,21.349394,-158.091434,
8,HI,Honolulu,15003010904,15003,15003010904,26.5,3570,21.409074,-157.748580,
9,HI,Honolulu,15003003900,15003,15003003900,25.2,655,21.302664,-157.861860,


In [26]:
# Now let's iterate thru that ONE state and get the data

for index,row in one_state.iterrows():

    try:
        lat = str(row["Lat"])
        lon = str(row["Lon"])
        query_url = url+"&lat="+lat+"&lon="+lon+"&wsapikey="+api_key
        walk_response = requests.get(query_url)
        walk_json = walk_response.json() 
        if walk_json["status"] == 41:
            print("Daily API quota exceeded")
            print("^^^^^^^^^^^^^^^^^^^^^^^^")
            break
            
        print("Processing LAT %s, LON, %s" % (lat, lon))
        one_state.loc[index, "Walk Score"] = walk_json["walkscore"]
    
    except:
        print("Data not found. Skipping")
        pass


Processing LAT 21.3388523084, LON, -157.875465156
Processing LAT 21.2655150514, LON, -157.817511576
Processing LAT 21.3199594239, LON, -157.864109557
Processing LAT 21.4040349086, LON, -157.991590779
Processing LAT 21.3120410065, LON, -157.834915593
Processing LAT 21.2713403723, LON, -157.792698935
Processing LAT 21.3940022356, LON, -158.124509059
Processing LAT 21.3493936019, LON, -158.091433922
Processing LAT 21.4090738111, LON, -157.748579917
Processing LAT 21.3026640114, LON, -157.861860063
Processing LAT 21.3785257575, LON, -157.743126514
Processing LAT 21.3350086973, LON, -158.103978266
Processing LAT 21.441822119, LON, -158.007985092
Processing LAT 21.3051406334, LON, -157.833155622
Processing LAT 21.2995998377, LON, -157.803133008
Processing LAT 21.289668402, LON, -157.828998268
Processing LAT 21.3128550435, LON, -158.031213771
Processing LAT 21.3951520649, LON, -157.784495187
Processing LAT 21.3942987901, LON, -157.87536044
Processing LAT 21.4363139061, LON, -157.921262424
Pro

In [27]:
one_state.head()

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,HI,Honolulu,15003006202,15003,15003006202,44.7,1701,21.338852,-157.875465,75
1,HI,Honolulu,15003001700,15003,15003001700,22.9,2437,21.265515,-157.817512,63
2,HI,Honolulu,15003005400,15003,15003005400,36.9,1637,21.319959,-157.864110,90
3,HI,Honolulu,15003008921,15003,15003008921,19.5,2668,21.404035,-157.991591,12
4,HI,Honolulu,15003003300,15003,15003003300,19.8,1132,21.312041,-157.834916,50


In [28]:
# Save output to CSV
output_csv = f'resources/{FindState}_complete.csv'
one_state.to_csv(output_csv, index=False)